In [10]:
import pandas as pd
import itertools
import random

In [4]:
# goal: create a train/test dataset using positive and negative pairs
df = pd.read_csv("../raw_data/tunes.csv")

In [ ]:
# Create positive pairs
positive_pairs = []
for tune_id, group in df.groupby('tune_id'):
    abc_list = group[['setting_id', 'abc']].values.tolist()
    # create one pair per tune (this seems like a decent enough way to start)
    if len(abc_list) > 1:
        pair = random.sample(abc_list, 2)
        positive_pairs.append((tune_id, tune_id, pair[0][0], pair[1][0], pair[0][1], pair[1][1]))


In [ ]:
# Create negative pairs - just the same amount as in the positive set
abc_list = df[['tune_id', 'setting_id', 'abc']].values.tolist()
negative_pairs = []
while len(negative_pairs) < len(positive_pairs):
    pair = random.sample(abc_list, 2)
    if pair[0][0] != pair[1][0]:
        negative_pairs.append((pair[0][0], pair[1][0], pair[0][1], pair[1][1], pair[0][2], pair[1][2]))

In [ ]:
# Create dataframes
positive_df = pd.DataFrame(positive_pairs, columns=['tune_a', 'tune_b', 'setting_a', 'setting_b', 'abc_a', 'abc_b'])
negative_df = pd.DataFrame(negative_pairs, columns=['tune_a', 'tune_b', 'setting_a', 'setting_b', 'abc_a', 'abc_b'])

In [28]:
positive_df.to_csv("../raw_data/positive_set.csv")

In [29]:
negative_df.to_csv("../raw_data/negative_set.csv")